In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, round
from pyspark.sql.window import Window

# Создаем SparkSession
spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()

# Создаем данные о продажах
data = [
    ("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)
]

# Создаем DataFrame
df = spark.createDataFrame(data, ["date", "category", "quantity", "revenue"])

# Выводим исходные данные
print("Исходные данные:")
df.show()

# 1. Рассчитываем среднюю выручку от продаж для каждой категории продукта по дням
# Определяем оконную спецификацию для категории и даты
window_spec = Window.partitionBy("category", "date")

# Применяем оконную функцию для расчета средней выручки
df_with_avg = df.withColumn("avg_revenue", round(avg("revenue").over(window_spec), 2))

print("Данные со средней выручкой:")
df_with_avg.show()

# 2. Используем pivot для преобразования данных
# Применяем group by по категории и затем pivot по дате
pivot_df = df.groupBy("category").pivot("date").agg(round(avg("revenue"), 2))

print("Результат после применения pivot:")
pivot_df.show()

Исходные данные:
+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+

Данные со средней выручкой:
+----------+-----------+--------+-------+-----------+
|      date|   category|quantity|revenue|avg_revenue|
+----------+-----------+--------+-------+-----------+
|2023-11-20|      Books|     150|   9000|     9000.0|
|2023-11-21|      Books|     200|  12000|    12000.0|
|2023-11-22|      Books|     180|  10000|    10000.0|
|2023-11-20|   Clothing|     300|  15000|    15000.0|
|2023-11-21|   Clothing|     2